In [14]:
import pandas as pd
import pickle
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
import itertools
import pyarrow.parquet as pq
from tqdm import tqdm

In [15]:
train = pq.read_table('train_without_None.gzip').to_pandas()

In [16]:
train

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type,user_id_filled
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy,951f53de61764ea0b51317200a0dbbfc
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,f5a2326a17484330aa8cb4019f1b1960
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,f5a2326a17484330aa8cb4019f1b1960
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,f5a2326a17484330aa8cb4019f1b1960
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,f5a2326a17484330aa8cb4019f1b1960
...,...,...,...,...,...,...,...
12292583,2022-08-11,1660246479,162851,0d3c58c6864546c689fa0997de484b30,18124998fd644e80b667a1d84bf63846,preview_click_response,18124998fd644e80b667a1d84bf63846
12292584,2022-08-11,1660246106,136818,ef559274a81e4ff4b92aa3e9f6805886,5c71c1f1b7cf492ba05871c400a05250,preview_click_response,5c71c1f1b7cf492ba05871c400a05250
12292585,2022-08-11,1660250139,143721,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response,33c968125459420e89ffbc8cffece317
12292586,2022-08-11,1660250318,154423,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response,33c968125459420e89ffbc8cffece317


### Список взаимодействий с вакансией

- 1) show_vacancy - просмотр вакансии
- 2) preview_click_vacancy - клик по карточке вакансии
- 3) **click_response - отклик со страницы вакансии**
- 4) **preview_click_response - отклик с карточки вакансии**
- 5) click_favorite - добавление вакансии в избранное  со страницы вакансии
- 6) preview_click_favorite - добавление вакансии в избраное с карточки вакансии
- 7) **click_contacts - клик на контакты со страницы вакансии**
- 8) **preview_click_contacts - клик на контакты из карточки вакансии**
- 9) **click_phone - клик на номер телефона, указанный в вакансии**
- 10) **preview_click_phone - клик на номер телефона из карточки вакансии**

Жирным выделены искомые "откликнется" и "позвонит". Меняем 1,2,5,6 - 1, остальное на 2

In [17]:
list1=['show_vacancy', 'preview_click_vacancy', 'click_favorite', 'preview_click_favorite']

list2=['click_contacts','preview_click_contacts','preview_click_phone','preview_click_response','click_phone','click_response']


In [18]:
mask1=train.iloc[:, 5:6].stack().apply(lambda x: x in list1).unstack()

mask2=train.iloc[:, 5:6].stack().apply(lambda x: x in list2).unstack()

In [19]:
mask1

,event_type
0,True
1,True
2,True
3,True
4,True
...,...
12292583,False
12292584,False
12292585,False
12292586,False


In [20]:
train.event_type[(mask1.event_type)]=1

C:\Users\Aliaksandr\AppData\Local\Temp\ipykernel_10500\2647336539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.event_type[(mask1.event_type)]=1


In [21]:
train.event_type[(mask2.event_type)]=10

C:\Users\Aliaksandr\AppData\Local\Temp\ipykernel_10500\3652181272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.event_type[(mask2.event_type)]=10


In [22]:
train=train[['user_id_filled', 'vacancy_id_', 'event_type']]

In [23]:
train=train.drop_duplicates()

In [24]:
train_group=train.groupby(['user_id_filled', 'vacancy_id_']).agg({'event_type':'max'})

In [25]:
train_group

event_type
user_id_filled                   vacancy_id_            
000089d26cdd49d68839c68bc10f2cf2 177380                1
                                 181353                1
                                 195890                1
                                 205562                1
                                 211064                1
...                                                  ...
ffffdb17f45b4032b386d691d52e6c00 107125               10
                                 107893               10
                                 150843               10
                                 160164               10
                                 185412               10

[4662690 rows x 1 columns]

In [26]:
train_group.to_pickle('train_group_add_cookie.pickle')